# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
filepath="../output_data/cities.csv"
cities_df=pd.read_csv(filepath)
cities_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Dingle,11.0,122.67,76.78,88,40,3.91,PH,1602941048
1,Port Hardy,50.7,-127.42,46.40,87,5,8.05,CA,1602941048
2,Hithadhoo,-0.6,73.08,83.91,75,99,18.86,MV,1602940870
3,Touros,-5.2,-35.46,87.80,52,75,18.34,BR,1602941048
4,Antalaha,-14.9,50.28,78.76,62,0,11.41,MG,1602941048


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
locations=cities_df[['Lat','Lng']]
weights=cities_df['Humidity']
# mid_map is the Lat and Lng coordinates so I could center the map on the screen.
mid_map=(0,15)
fig=gmaps.figure(center=mid_map,zoom_level=1)
heat_layer=gmaps.heatmap_layer(locations,weights=weights)
heat_layer.max_intensity=100
heat_layer.point_radius=5
heat_layer.dissipation=False
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# I included humidity level as well as the temp, cloudiness and windspeed.
hitemp=80
lwtemp=70
humidity=60
cloudiness=25
windspeed=10
my_spots_df=cities_df.loc[
                    (cities_df['Max Temp']>=lwtemp) & (cities_df['Max Temp']<=hitemp) &
                    (cities_df['Humidity']<=humidity) &
                    (cities_df['Cloudiness']<=cloudiness) & 
                    (cities_df['Wind Speed']<=windspeed) 
                    ,:]
my_spots_df              

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
63,Santiago,-33.46,-70.65,71.01,59,0,2.24,CL,1602941031
264,Ribeira Brava,32.65,-17.07,73.40,60,20,5.82,PT,1602941075
320,Ponta do Sol,32.67,-17.10,73.40,60,20,5.82,PT,1602941080
404,Kruisfontein,-34.00,24.73,75.00,60,18,8.01,ZA,1602941088
434,Santa Fe,-31.63,-60.70,79.00,54,12,4.00,AR,1602941090
478,Garmsār,35.22,52.34,73.71,17,0,2.93,IR,1602941095
493,Bāfq,31.61,55.41,74.44,12,0,7.96,IR,1602941096
541,Santa Maria,-29.68,-53.81,74.23,52,7,6.67,BR,1602941080


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
#Limit lookups to 10 vacation spots.
lookup_limit=10
hotel_df=my_spots_df.copy()
hotel_df['Hotel Name']=''

params={
        "query":'hotel',
        "radius":5000,
        "type":"lodging",
        "language":"en",
        "key":g_key
}

base_url="https://maps.googleapis.com/maps/api/place/nearbysearch/json"
x=0
for index, row in hotel_df.iterrows():
    if (x==lookup_limit):
        break
    x+=1
    lat=row['Lat']
    lng=row['Lng']
    params['location']=f"{lat},{lng}"
    hotel_names=requests.get(base_url,params=params).json()
    try:
        hotel_df.loc[index,"Hotel Name"]=hotel_names["results"][0]["name"]
    except:
        hotel_df.loc[index,"Hotel Name"]="No Hotel"
hotel_df.head(lookup_limit)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
63,Santiago,-33.46,-70.65,71.01,59,0,2.24,CL,1602941031,Hotel Panamericano
264,Ribeira Brava,32.65,-17.07,73.40,60,20,5.82,PT,1602941075,Hotel do Campo
320,Ponta do Sol,32.67,-17.10,73.40,60,20,5.82,PT,1602941080,Hotel do Campo
404,Kruisfontein,-34.00,24.73,75.00,60,18,8.01,ZA,1602941088,Oyster Bay House Rental
434,Santa Fe,-31.63,-60.70,79.00,54,12,4.00,AR,1602941090,Conquistador Hotel
478,Garmsār,35.22,52.34,73.71,17,0,2.93,IR,1602941095,Peyvand Hotel
493,Bāfq,31.61,55.41,74.44,12,0,7.96,IR,1602941096,Alamdar Hotel
541,Santa Maria,-29.68,-53.81,74.23,52,7,6.67,BR,1602941080,Hotel Morotin


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
# Add marker layer ontop of heat map
marker_layer=gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))